<a href="https://colab.research.google.com/github/jaejunchoe/HAIDS-Lab/blob/main/utils.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import time
import pandas as pd
import torch
from torch.utils.data import Dataset
from torch.nn import functional as F

def date(f='%Y-%m-%d %H:%M:%S'):
    return time.strftime(f, time.localtime())

# glove.6B.50d.txt 파일 load하고 embedding하는 함수
def load_embedding(word2vec_file):
    with open(word2vec_file, encoding='utf-8') as f:
        word_emb = [[0]]
        word_dict = {'<UNK>': 0}
        for line in f.readlines():
            tokens = line.split(' ')
            word_emb.append([float(i) for i in tokens[1:]])
            word_dict[tokens[0]] = len(word_dict)
        word_emb[0] = [0] * len(word_emb[1])
    return word_emb, word_dict


# MSE 계산하는 함수 -> train과 valid에서 사용
def predict_mse(model, dataloader, device):
    mse, sample_count = 0, 0
    with torch.no_grad():
        for batch in dataloader:
            user_reviews, item_reviews, ratings = map(lambda x: x.to(device), batch)
            predict = model(user_reviews, item_reviews)
            mse += F.mse_loss(predict, ratings, reduction='sum').item()
            sample_count += len(ratings)
    return mse / sample_count

# Pytorch의 Dataset 클래스를 확장해서 DeepCoNN 모델의 학습에 적합한 데이터셋을 생성하는 Class
class DeepCoNNDataset(Dataset):

    ## csv 파일을 읽고 모델 입력 형식에 맞게 바꾸는 함수
    def __init__(self, data_path, word_dict, config, retain_rui=True):
        self.word_dict = word_dict
        self.config = config
        self.retain_rui = retain_rui                        # 리뷰 포함에 대한 조건
        self.PAD_WORD_idx = self.word_dict[config.PAD_WORD]
        self.review_length = config.review_length
        self.review_count = config.review_count
        self.lowest_r_count = config.lowest_review_count

        df = pd.read_csv(data_path, header=None, names=['userID', 'itemID', 'review', 'rating'])
        df['review'] = df['review'].apply(self._review2id)
        self.sparse_idx = set()

        user_reviews = self._get_reviews(df)
        item_reviews = self._get_reviews(df, 'itemID', 'userID')
        rating = torch.Tensor(df['rating'].to_list()).view(-1, 1)

        # 희소 데이터(self.sparse_idx)를 제외하고 필터 -> lowest_review_count = 0이기에 손실되는 데이터 없을 것
        self.user_reviews = user_reviews[[idx for idx in range(user_reviews.shape[0]) if idx not in self.sparse_idx]]
        self.item_reviews = item_reviews[[idx for idx in range(item_reviews.shape[0]) if idx not in self.sparse_idx]]
        self.rating = rating[[idx for idx in range(rating.shape[0]) if idx not in self.sparse_idx]]


    ## 데이터셋에서 특정 idx로 데이터를 반환함 -> (user_reviews, item_reviews, rating)
    def __getitem__(self, idx):
        return self.user_reviews[idx], self.item_reviews[idx], self.rating[idx]


    ## 데이터셋 전체 길이 반환
    def __len__(self):
        return self.rating.shape[0]


    ## 사용자 또는 아이템 단위로 리뷰 데이터를 그룹화하고, 최대 리뷰 개수(10개)에 따라 정리
    def _get_reviews(self, df, lead='userID', costar='itemID'):
        reviews_by_lead = dict(list(df[[costar, 'review']].groupby(df[lead])))
        lead_reviews = []
        for idx, (lead_id, costar_id) in enumerate(zip(df[lead], df[costar])):
            df_data = reviews_by_lead[lead_id]


            # self.retain_rui = True: 사용자가 작성한 모든 리뷰를 가져와
            # true이기에 else가 실행되지않을 것 -> reviews = df_data['review'].to_list()가 이미 모든 리뷰를 포함하기 때문
            reviews = df_data['review'].to_list() if self.retain_rui else df_data['review'][df_data[costar] != costar_id].to_list()


            if len(reviews) < self.lowest_r_count:
                self.sparse_idx.add(idx)
            reviews = self._adjust_review_list(reviews, self.review_length, self.review_count)      # 개수와 길이 조정
            lead_reviews.append(reviews)
        return torch.LongTensor(lead_reviews)


    ## 리뷰 데이터를 고정된 리뷰 수(review_count)와 리뷰 길이(review_length)로 조정
    def _adjust_review_list(self, reviews, r_length, r_count):
        reviews = reviews[:r_count] + [[self.PAD_WORD_idx] * r_length] * (r_count - len(reviews))
        reviews = [r[:r_length] + [0] * (r_length - len(r)) for r in reviews]
        return reviews

    ## 리뷰 문자열을 단어 임베딩 인덱스 리스트로 변환
    def _review2id(self, review):
        if not isinstance(review, str):
            return []
        return [self.word_dict.get(word, self.PAD_WORD_idx) for word in review.split()]